In [1]:
import os
from pathlib import Path
import requests
import zipfile
import gdown
import math
import copy
import random
import scipy.io

import numpy as np
import mne
from mne.preprocessing import ICA
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

# Torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset, random_split

# Scikit-Learn
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

c:\Users\lsg-yy\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# [1] EEG 데이터 다운로드

url = "https://www.bbci.de/competition/download/competition_iv/BCICIV_2a_gdf.zip"
output = "BCICIV_2a_gdf.zip"

r = requests.get(url)
with open(output, "wb") as f:
    f.write(r.content)

print("Download completed:", output)

os.makedirs("cleaned_data", exist_ok=True)
os.makedirs("cleaned_data/first_session", exist_ok=True)
os.makedirs("cleaned_data/second_session", exist_ok=True)

os.makedirs("raw_data", exist_ok=True)

with zipfile.ZipFile("BCICIV_2a_gdf.zip", "r") as z:
    z.extractall("raw_data")
print("Extraction completed: ", output)

shareable_link = 'https://drive.google.com/file/d/11Ke2Xta1kv2xu2Mybuu_X51zJYjQ-VFo/view?usp=drive_link'

file_id = shareable_link.split('/d/')[1].split('/view')[0]

download_url = f'https://drive.google.com/uc?id={file_id}&export=download'

output_file = 'true_labels.zip'

# Download the file
gdown.download(download_url, output_file, quiet=False)
print("Download completed: ", output_file)

os.makedirs("second_session_labels", exist_ok=True)

with zipfile.ZipFile("true_labels.zip", 'r') as z:
    z.extractall("second_session_labels")
print("Extraction completed: ", output_file)



Download completed: BCICIV_2a_gdf.zip
Extraction completed:  BCICIV_2a_gdf.zip


Downloading...
From: https://drive.google.com/uc?id=11Ke2Xta1kv2xu2Mybuu_X51zJYjQ-VFo&export=download
To: h:\EEG project\Project\true_labels.zip
100%|██████████| 7.20k/7.20k [00:00<?, ?B/s]

Download completed:  true_labels.zip
Extraction completed:  true_labels.zip


In [3]:
# [2] First Session 전처리 및 라벨 병합: 전 피험자 루프 + ICA 옵션 + 22채널 EEG

raw_data_folder = "raw_data"
cleaned_data_folder = "cleaned_data/first_session"
os.makedirs(cleaned_data_folder, exist_ok=True)

# ICA 사용 옵션 (기본 True)
USE_ICA = True

# BCI Competition IV 2a 표준 EEG 22채널 순서
target_22 = [
    "Fz", "FC3", "FC1", "FCz", "FC2", "FC4",
    "C5", "C3", "C1", "Cz", "C2", "C4",
    "C6", "CP3", "CP1", "CPz", "CP2", "CP4",
    "P1", "Pz", "P2", "POz"
]

# 첫 세션 학습 파일 (예: A01T.gdf ~ A09T.gdf)
gdf_files = sorted(
    f for f in os.listdir(raw_data_folder)
    if f.endswith("T.gdf")
)

epochs_list = []

for fname in gdf_files:
    file_path = os.path.join(raw_data_folder, fname)
    subject_id = Path(fname).stem  # 예: "A01T"

    # 1) Raw 데이터 로드
    raw = mne.io.read_raw_gdf(file_path, preload=True)

    # 2) 채널 타입 설정(EOG) 및 기본 검증
    ch_names = raw.info["ch_names"]
    if len(ch_names) != 25:
        raise RuntimeError(f"{subject_id}: 채널 수가 25가 아님 (len={len(ch_names)})")

    raw.set_channel_types({
        "EOG-left": "eog",
        "EOG-central": "eog",
        "EOG-right": "eog"
    })

    # 3) EEG 22채널 표준 이름 매핑 (0~21: EEG, 22~24: EOG 유지)
    mapping = {}
    for i, name in enumerate(ch_names):
        if i < 22:  # EEG
            mapping[name] = target_22[i]
        else:       # EOG
            mapping[name] = name
    mne.rename_channels(raw.info, mapping)

    # 4) 공통 평균 참조(CAR) + 표준 10-20 몽타주 적용
    raw.set_eeg_reference("average", projection=False)
    montage = mne.channels.make_standard_montage("standard_1020")
    raw.set_montage(montage, on_missing="ignore")

    # 5) 필터링: 노치(50/100 Hz) + 대역통과(8–30 Hz)
    raw.notch_filter(freqs=[50.0, 100.0], picks=["eeg", "eog"], fir_design="firwin")
    raw.filter(l_freq=8.0, h_freq=30.0, picks=["eeg", "eog"], fir_design="firwin")

    # 6) ICA 기반 EOG 아티팩트 제거(옵션)
    if USE_ICA:
        ica = ICA(n_components=22, random_state=97, max_iter="auto")
        ica.fit(raw.copy().pick("eeg"))

        eog_inds, _ = ica.find_bads_eog(
            raw,
            ch_name=["EOG-left", "EOG-central", "EOG-right"]
        )
        ica.exclude = eog_inds

        raw_clean = raw.copy()
        ica.apply(raw_clean)
    else:
        raw_clean = raw

    # 7) 최종 EEG 22채널만 선택
    raw_eeg = raw_clean.copy().pick_types(eeg=True)
    if raw_eeg.info["nchan"] != 22:
        raise RuntimeError(
            f"{subject_id}: EEG 채널 수가 22가 아님 (nchan={raw_eeg.info['nchan']})"
        )

    # 8) 이벤트 → motor imagery epoch (0~4 s)
    event_id = {"769": 1, "770": 2, "771": 3, "772": 4}  # Left/Right/Feet/Tongue
    events, _ = mne.events_from_annotations(raw_eeg, event_id=event_id)

    epochs = mne.Epochs(
        raw_eeg,
        events,
        event_id=event_id,
        tmin=0.0,
        tmax=4.0,
        baseline=None,
        preload=True
    )

    # 9) 개별 피험자 epochs 저장 (.fif, A01T.fif 등)
    subj_fif_path = os.path.join(cleaned_data_folder, fname[:-4] + ".fif")
    epochs.save(subj_fif_path, overwrite=True)

    epochs_list.append(epochs)

# 10) 전체 피험자 epochs 결합 및 저장
all_epochs = mne.concatenate_epochs(epochs_list)
all_fif_path = os.path.join(cleaned_data_folder, "First_Session_Subjects.fif")
all_epochs.save(all_fif_path, overwrite=True)

first_session_data = all_epochs.get_data()
first_session_labels = all_epochs.events[:, -1]

print("First_session_dataset shape:", first_session_data.shape)
print("First_session_labels shape:", first_session_labels.shape)


Extracting EDF parameters from h:\EEG project\Project\raw_data\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


c:\Users\lsg-yy\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwi

C:\Users\lsg-yy\AppData\Local\Temp\ipykernel_13392\2847768267.py:65: RuntimeWarning: Using n_components=22 (resulting in n_components_=22) may lead to an unstable mixing matrix estimation because the ratio between the largest (11) and smallest (1.1e-28) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 21
  ica.fit(raw.copy().pick("eeg"))


Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

... filte

...

... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 sam

C:\Users\lsg-yy\AppData\Local\Temp\ipykernel_13392\2847768267.py:101: RuntimeWarning: This filename (cleaned_data/first_session\A09T.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(subj_fif_path, overwrite=True)


Not setting metadata
2592 matching events found
No baseline correction applied
Overwriting existing file.


C:\Users\lsg-yy\AppData\Local\Temp\ipykernel_13392\2847768267.py:106: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  all_epochs = mne.concatenate_epochs(epochs_list)
C:\Users\lsg-yy\AppData\Local\Temp\ipykernel_13392\2847768267.py:108: RuntimeWarning: This filename (cleaned_data/first_session\First_Session_Subjects.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  all_epochs.save(all_fif_path, overwrite=True)


Overwriting existing file.
Overwriting existing file.
First_session_dataset shape: (2592, 22, 1001)
First_session_labels shape: (2592,)


In [4]:
# [3] Second Session 전처리 및 라벨 병합

raw_data_folder = "raw_data"
cleaned_data_folder = "cleaned_data/second_session"
mat_folder = "second_session_labels"

os.makedirs(cleaned_data_folder, exist_ok=True)

USE_ICA = True  # ICA 사용 여부

# 표준 EEG 22채널 이름 순서
target_22 = [
    "Fz", "FC3", "FC1", "FCz", "FC2", "FC4",
    "C5", "C3", "C1", "Cz", "C2", "C4",
    "C6", "CP3", "CP1", "CPz", "CP2", "CP4",
    "P1", "Pz", "P2", "POz"
]

# 두 번째 세션 파일(*E.gdf)
files = sorted(
    f for f in os.listdir(raw_data_folder)
    if f.endswith("E.gdf")
)

raw_list = []
second_session_labels = np.array([], dtype=int)

for file in files:
    file_path = os.path.join(raw_data_folder, file)
    subject_id = Path(file).stem

    # 1) Raw 로드
    raw = mne.io.read_raw_gdf(file_path, preload=True)

    # 2) 채널 타입(EOG) 지정
    ch_names = raw.info["ch_names"]
    raw.set_channel_types({
        "EOG-left": "eog",
        "EOG-central": "eog",
        "EOG-right": "eog"
    })

    # 3) EEG 22채널 이름 표준화
    mapping = {}
    for i, name in enumerate(ch_names):
        mapping[name] = target_22[i] if i < 22 else name
    mne.rename_channels(raw.info, mapping)

    # 4) CAR + 몽타주
    raw.set_eeg_reference("average", projection=False)
    montage = mne.channels.make_standard_montage("standard_1020")
    raw.set_montage(montage, on_missing="ignore")

    # 5) 필터링
    raw.notch_filter(freqs=[50, 100], picks=["eeg", "eog"], fir_design="firwin")
    raw.filter(l_freq=8, h_freq=30, picks=["eeg", "eog"], fir_design="firwin")

    # 6) ICA 옵션 반영
    if USE_ICA:
        ica = ICA(n_components=22, random_state=97, max_iter="auto")
        ica.fit(raw.copy().pick("eeg"))
        eog_inds, _ = ica.find_bads_eog(
            raw, ch_name=["EOG-left", "EOG-central", "EOG-right"]
        )
        ica.exclude = eog_inds
        raw_clean = raw.copy()
        ica.apply(raw_clean)
    else:
        raw_clean = raw

    # 7) EEG 22채널만 선택
    raw_eeg = raw_clean.copy().pick_types(eeg=True)

    # 8) 개별 피험자 전처리 결과 저장
    new_file_path = os.path.join(cleaned_data_folder, file[:-4] + ".fif")
    raw_eeg.save(new_file_path, overwrite=True)

    raw_list.append(raw_eeg)

    # 9) .mat 라벨 병합
    mat_file_name = file.replace(".gdf", ".mat")
    mat_path = os.path.join(mat_folder, mat_file_name)

    if os.path.exists(mat_path):
        mat_data = scipy.io.loadmat(mat_path)
        class_labels = mat_data.get("classlabel", [])
        if class_labels.size > 0:
            second_session_labels = np.concatenate(
                (second_session_labels, np.array(class_labels).flatten())
            )

# 10) 전체 피험자 raw 결합
final_raw = mne.concatenate_raws(raw_list)
new_file_path = os.path.join(cleaned_data_folder, "Second_Session_Subjects.fif")
final_raw.save(new_file_path, overwrite=True)

# 11) 이벤트 → 에포크 생성
events = mne.events_from_annotations(final_raw)
print(events[1])

epochs = mne.Epochs(
    final_raw,
    events[0],
    event_id=7,
    tmin=0,
    tmax=4,
    reject=None,
    baseline=None,
    preload=None
)

second_session_data = epochs.get_data(copy=True)


print("Second_Session_Dataset shape:", second_session_data.shape)
print("Second_session_labels shape:", second_session_labels.shape)


Extracting EDF parameters from h:\EEG project\Project\raw_data\A01E.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 686999  =      0.000 ...  2747.996 secs...


c:\Users\lsg-yy\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwi

C:\Users\lsg-yy\AppData\Local\Temp\ipykernel_13392\2197358156.py:61: RuntimeWarning: Using n_components=22 (resulting in n_components_=22) may lead to an unstable mixing matrix estimation because the ratio between the largest (11) and smallest (2.3e-28) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 21
  ica.fit(raw.copy().pick("eeg"))


... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 sam

...

... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 sam

C:\Users\lsg-yy\AppData\Local\Temp\ipykernel_13392\2197358156.py:76: RuntimeWarning: This filename (h:\EEG project\Project\cleaned_data\second_session\A09E.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_eeg.save(new_file_path, overwrite=True)


Closing h:\EEG project\Project\cleaned_data\second_session\A09E.fif
[done]
Overwriting existing file.
Writing h:\EEG project\Project\cleaned_data\second_session\Second_Session_Subjects.fif
Overwriting existing file.


C:\Users\lsg-yy\AppData\Local\Temp\ipykernel_13392\2197358156.py:95: RuntimeWarning: This filename (h:\EEG project\Project\cleaned_data\second_session\Second_Session_Subjects.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  final_raw.save(new_file_path, overwrite=True)


Closing h:\EEG project\Project\cleaned_data\second_session\Second_Session_Subjects.fif
[done]
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '783']
{'1023': 1, '1072': 2, '276': 3, '277': 4, '32766': 5, '768': 6, '783': 7}
Not setting metadata
2592 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 2592 events and 1001 original time points ...
0 bad epochs dropped
Second_Session_Dataset shape: (2592, 22, 1001)
Second_session_labels shape: (2592,)


### Structuring Data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Loss Function
criterion = nn.CrossEntropyLoss()

# Normalizing Labels to [0, 1, 2, 3]
y_train = first_session_labels - np.min(first_session_labels)
y_test = second_session_labels - np.min(second_session_labels)

# Normalizing Input features: z-score(mean=0, std=1)
X_first_session = (first_session_data - np.mean(first_session_data)) / np.std(first_session_data)
X_second_session = (second_session_data - np.mean(second_session_data)) / np.std(second_session_data)

X = np.concatenate((X_first_session, X_second_session))
y = np.concatenate((y_train, y_test))

# Spliting  Data: 90% for Train and 10% for Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

# Converting to Tensor
X_train = torch.Tensor(X_train).unsqueeze(1).to(device)
X_test = torch.Tensor(X_test).unsqueeze(1).to(device)
y_train = torch.LongTensor(y_train).to(device)
y_test = torch.LongTensor(y_test).to(device)

# Creating Tensor Dataset
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Printing the sizes
print("Size of X_train:", X_train.size())
print("Size of X_test:", X_test.size())
print("Size of y_train:", y_train.size())
print("Size of y_test:", y_test.size())


cuda
Size of X_train: torch.Size([4665, 1, 22, 1001])
Size of X_test: torch.Size([519, 1, 22, 1001])
Size of y_train: torch.Size([4665])
Size of y_test: torch.Size([519])
